# 0. 사용방법

1. shift + enter
2. 원안의 화살표 버튼 클릭

위의 방법중 하나를 선택하셔서, 순차적으로 다운로드하시면 됩니다

# 1. PIP 다운로드

In [ ]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


# 2. 파일 업로드

bom.excel 파일을 txt 형식으로 변환해서 올리시면됩니다.

In [ ]:
from google.colab import files
from IPython.display import clear_output


uploaded = files.upload()

for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')


Saving console_26.txt to console_26.txt
User uploaded file "console_26.txt" with length 1937 bytes


# 3. 엑셀 파일 생성

**V2.2 업데이트 항목**

1) capaciotr, resistor => chip res, chip cap 변경

2) 파일이름, bom 제목, txt 기준으로 자동 변경

3) 시간 추가(예정)

4) 프롬프트 수정 : Strength -> Capacitance



---


**업데이트 예정 목록**

1) 3번 row 길이 늘리기.

2) subtitle 크기 늘리기

3) 생산수량 및 갯수 글자 크기 늘리기

4) Rev : (num)  추가하기

5) TXT 파일 인식 안될 때, 수동으로 업로드 하는 방식 추가

In [ ]:
import anthropic
import csv
import os
import pandas as pd
import google.colab as output

# Anthropic API 키 설정
client = anthropic.Client(api_key="Claude API 키는 회원가입 후 얻을 수 있습니다.")
# CSV 파일 읽기
def read_csv(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        return list(reader)

# Claude API로 질문 보내기
def ask_claude(csv_content, question):
    prompt = f"""Here's the content of the CSV file:

{csv_content}

Question: {question}

Please answer the question based on the CSV content."""

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=2000,
        temperature=0,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.content[0].text

# 메인 실행 코드
if __name__ == "__main__":
    csv_file_path = rf"/content/{filename}"
    # CSV 파일 경로를 지정하세요
    csv_content = read_csv(csv_file_path)

    #제목 추출
    extract_sub = csv_content[0][0].split('Revised')
    extract_sub = extract_sub[0]

    # CSV 내용을 문자열로 변환
    csv_str = "\n".join([",".join(row) for row in csv_content])

    question = """
    THIS IS BOM OF PCB. Classify by category, specification, reference, and quantity.
    Properly distinguish between beads and inductors.
    Convert it to a format that can be used in Excel files.
    The order is first capacitor, second resistor, in ascending order of Capacitance.
    Add each classified category name to the column.
    Just give me the data without any explanation.
    """  # 원하는 질문으로 변경하세요

    answer = ask_claude(csv_str, question)
    print("Claude의 답변:", answer)
import pandas as pd
import io
import re
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.datavalidation import DataValidation
from datetime import date
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment, Color, Border, Side


#제목 추출
extract_sub = read_csv(fr"/content/{filename}")
extract_sub = extract_sub[0][0].split('Revised')
extract_sub = extract_sub[0]
extract_sub = extract_sub.replace(" ", "_")

# CSV 데이터를 DataFrame으로 변환
print(answer)

# Attempt to read the CSV, handling potential errors
try:
    df = pd.read_csv(io.StringIO(answer))
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
    # Print the problematic lines for debugging
    for i, line in enumerate(answer.split('\n')):
        if i >= 13:  # Print lines around line 15
            print(f"Line {i+1}: {line}")
    # Optionally, you can try to manually fix the CSV data here before reading it again.

# 새 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "BOM"

# 제목 추가
ws['B1'] = f"{extract_sub}"
ws['B1'].font = Font(bold=True, size=16)
ws.merge_cells('B1:F1')

# 수량 입력 칸 추가
ws['G1'] = "생산수량:"
ws['H1'] = 1  # 기본값 설정
ws['G1'].font = Font(bold=True)
ws['H1'].font = Font(bold=True)
ws['B1'].alignment = Alignment(horizontal='center', vertical='center')  # B1 셀 가운데 정렬
ws['H1'].font = Font(bold=True, color="FF0000")  # 빨간색 설정
ws['H1'].alignment = Alignment(horizontal='center', vertical='center')  # H1 셀 가운데 정렬
ws['G1'].alignment = Alignment(horizontal='center', vertical='center')  # G1 셀 가운데 정렬

# 날짜 추가
today = date.today().strftime("%Y-%m-%d")
ws['B2'] = f"Date: {today}"
ws['B2'].font = Font(italic=True, bold=True)

default_font = Font(bold=True)

# 헤더 추가
headers = ["No", "Category", "Specification", "Package(mm)", "Reference", "Quantity", "Total Qt", "In Stock", "Required", "Remarks"]
for c, header in enumerate(headers, start=2):
    ws.cell(row=4, column=c, value=header)

# 데이터프레임을 워크시트에 쓰기 (5행부터 시작)
for r, row in enumerate(df.values, start=5):
    no_cell = ws.cell(row=r, column=2, value=r-4)
    no_cell.font = default_font
    for c, value in enumerate(row, start=3):
        if c < 5:  # Package 열 이전까지는 그대로 입력
            data_cell = ws.cell(row=r, column=c, value=value)
            data_cell.font = default_font
        elif c == 5:  # Package 열에 빈 값 입력
            data_cell = ws.cell(row=r, column=c, value="")
            data_cell.font = default_font
            reference_cell = ws.cell(row=r, column=c+1, value=value)
            reference_cell.font = default_font
        else:  # 나머지 열들도 한 칸씩 뒤로
            data_cell = ws.cell(row=r, column=c+1, value=value)
            data_cell.font = default_font

# 같은 카테고리 셀 병합
current_category = None
start_row = 5

for row in range(5, ws.max_row + 1):
    category = ws.cell(row=row, column=3).value
    if category != current_category:
        if current_category is not None:
            # 이전 카테고리 셀 병합
            if start_row < row - 1:
                ws.merge_cells(f'C{start_row}:C{row-1}')
        current_category = category
        start_row = row

# 마지막 카테고리 셀 병합
if start_row < ws.max_row:
    ws.merge_cells(f'C{start_row}:C{ws.max_row}')

# 병합된 셀의 정렬 조정
for merged_range in ws.merged_cells.ranges:
    if merged_range.min_col == 3:  # C열(카테고리 열)
        ws.cell(row=merged_range.min_row, column=3).alignment = Alignment(horizontal='center', vertical='center')

# Total Quantity 계산 공식 추가
for row in range(5, len(df) + 5):
    quantity_cell = ws.cell(row=row, column=7)
    total_quantity_cell = ws.cell(row=row, column=8)
    total_quantity_cell.value = f"={quantity_cell.coordinate}*$H$1"
    total_quantity_cell.font = Font(bold=True)  # 굵은 글씨 적용


# Required 계산 공식 추가
for row in range(5, len(df) + 5):
    total_quantity_cell = ws.cell(row=row, column=8)
    in_stock_cell = ws.cell(row=row, column=9)
    required_cell = ws.cell(row=row, column=10)
    required_cell.value = f"=MAX(0, {total_quantity_cell.coordinate}-{in_stock_cell.coordinate})"
    required_cell.font = Font(bold=True)  # 굵은 글씨 적용

# 스타일 설정 (변경된 부분)
header_font = Font(bold=True)
header_fill = PatternFill(start_color="B4C6E7", end_color="B4C6E7", fill_type="solid")
#thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
#thick_border = Border(left=Side(style='thick'), right=Side(style='thick'), top=Side(style='thick'), bottom=Side(style='thick'))

category_colors = {
    "Resistor": "FFD9D9",      # 연한 빨강
    "Capacitor": "D9FFD9",     # 연한 초록
    "Diode": "FFE6B3",         # 연한 주황
    "Transistor": "B3E6FF",    # 연한 파랑
    "Crystal": "E6B3FF",       # 연한 보라
    "LED": "FFFFB3",           # 연한 노랑
    "Inductor": "FFB3D9",      # 연한 분홍
    "Bead": "D9E6FF",          # 매우 연한 파랑
    "ESD Protection": "E6FFB3", # 연한 라임
    "IC": "B3FFE6",            # 연한 민트
    "Connector": "FFD9B3",     # 연한 복숭아
    "Switch": "D9FFE6",        # 연한 청록
    "Buzzer": "E6D9FF",        # 연한 라벤더
    "Test Point": "FFE6E6",  # 매우 연한 분홍
    "TP": "FFE6E6",  # 매우 연한 분홍,
    "Others": "E6E6E6"         # 연한 회색
}


# 스타일 적용 (변경된 부분)
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=ws.max_column):
    for cell in row:
        thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

        thick_border = Border(left=Side(style='thick'),
                      right=Side(style='thick'),
                      top=Side(style='thick'),
                      bottom=Side(style='thick'))
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center', vertical='center')

        if cell.row == 4 or cell.column == 2:  # 헤더 행이거나 No 열인 경우
            cell.font = header_font
            cell.fill = header_fill
        else:
            category = ws.cell(row=cell.row, column=3).value
            if category in category_colors:
                fill_color = category_colors[category]
                cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")

            if cell.column in [4, 5, 6]:  # D열, E열, F열 정렬 해제
                cell.alignment = Alignment(horizontal='left', vertical='center')

# A열의 스타일 제거
for cell in ws['A']:
    cell.border = None
    cell.fill = PatternFill(fill_type=None)

# 외곽 테두리 두껍게 설정 (새로 추가된 부분)
last_row = ws.max_row
last_col = ws.max_column

for col in range(2, last_col + 1):
    ws.cell(row=4, column=col).border = Border(top=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), bottom=Side(style='thin'))
    ws.cell(row=last_row, column=col).border = Border(bottom=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'))

for row in range(4, last_row + 1):
    ws.cell(row=row, column=2).border = Border(left=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), right=Side(style='thin'))
    ws.cell(row=row, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'))

# 모서리 셀 테두리 수정 (새로 추가된 부분)
ws.cell(row=4, column=2).border = Border(left=Side(style='thick'), top=Side(style='thick'), right=Side(style='thin'), bottom=Side(style='thin'))
ws.cell(row=4, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thick'), left=Side(style='thin'), bottom=Side(style='thin'))
ws.cell(row=last_row, column=2).border = Border(left=Side(style='thick'), bottom=Side(style='thick'), right=Side(style='thin'), top=Side(style='thin'))
ws.cell(row=last_row, column=last_col).border = Border(right=Side(style='thick'), bottom=Side(style='thick'), left=Side(style='thin'), top=Side(style='thin'))

# 열 너비 조정
width = 10  # 기본 너비
for column in ws.columns:
    max_length = 0
    column_letter = get_column_letter(column[0].column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column_letter].width = adjusted_width
    ws.column_dimensions['B'].width = 6  # A열의 너비를 6으로 설정
    ws.column_dimensions['F'].width = 40  # E열의 너비를 40으로 설정
    ws.column_dimensions['G'].width = width  # F열의 너비를 10으로 설정
    ws.column_dimensions['H'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['I'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['J'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['K'].width = 25  # J열의 너비를 16으로 설정


# 'resister' 문자열을 찾아 'chip capacitor'로 변경
for row in ws.iter_rows():
    for cell in row:
        if cell.value == 'Resistor':
            cell.value = 'Chip Res'
        if cell.value == 'Capacitor':
            cell.value = 'Chip Cap'


# 특수 문자를 언더스코어로 대체
extract_sub = re.sub(r'[<>:"/\\|?*&]', '_', extract_sub)

# Production Quantity 셀에 데이터 유효성 검사 추가
dv = DataValidation(type="whole", operator="greaterThan", formula1=0)
dv.error = "Please enter a positive integer"
dv.errorTitle = "Invalid Input"
ws.add_data_validation(dv)
dv.add('H1')

# 파일 저장
excel_file = f'{extract_sub}.xlsx'
wb.save(excel_file)

print(f"Excel 파일이 생성되었습니다: {extract_sub}xlsx")

Claude의 답변: Category,Specification,Reference,Quantity
Capacitor,8pF,"C2,C3",2
Capacitor,20pF,C21,1
Capacitor,22pF,"C4,C5",2
Capacitor,24pF,"C29,C30,C31,C32,C33",5
Capacitor,27pF,C34,1
Capacitor,0.1uF,"C1,C7,C8,C9,C11,C12,C13,C14,C15,C16,C17,C18,C26,C35,C39,C40",16
Capacitor,100nF,C20,1
Capacitor,1uF,"C19,C25,C36",3
Capacitor,2.2uF,C10,1
Capacitor,4.7uF,"C37,C38",2
Capacitor,10uF,C6,1
Capacitor,10uF/16V/2012,C24,1
Capacitor,22uF/10V/2012,"C22,C23",2
Capacitor,47uF/10V/3216,"C27,C28",2
Resistor,0,"R15,R17,R55,R76,R83,R111,R112",7
Resistor,22,"R32,R33,R35,R38,R40,R42,R46,R50,R51,R52,R54,R56,R58,R60,R62,R64,R66,R68,R70,R71,R84,R85",22
Resistor,24/3216,R82,1
Resistor,27R,"R72,R73",2
Resistor,100,"R20,R21,R22,R23,R24,R25,R86,R87,R88",9
Resistor,200,"R9,R10,R11,R12,R14,R39,R41",7
Resistor,1K,"R28,R36,R97,R98,R99,R100,R101",7
Resistor,1.5K,R30,1
Resistor,2K,"R8,R45,R53,R74,R79,R81",6
Resistor,3K,R78,1
Resistor,4.7K,"R1,R2,R3,R4,R5,R6,R7,R31",8
Resistor,10K,"R34,R37,R48,R57,R59,R61,R89,R90,R91,

2번째 방법

In [ ]:
import pandas as pd
import io
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from datetime import date
import os
from google.colab import files
from IPython.display import clear_output

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 처리
for file_name, file_content in uploaded.items():
    print(f'Processing file "{file_name}" with length {len(file_content)} bytes')

    # 파일 내용을 문자열로 디코딩
    txt_data = file_content.decode('utf-8')

    # 텍스트 데이터를 DataFrame으로 변환
    lines = txt_data.strip().split('\n')
    data = []
    header_found = False
    for line in lines:
        parts = line.strip().split('\t')
        if len(parts) == 4 and parts[0].lower() == 'item':
            header_found = True
            continue
        if header_found and len(parts) == 4:
            item, quantity, reference, part = parts
            if item and quantity and reference and part:  # 모든 필드가 비어있지 않은 경우만 추가
                data.append([part, reference, quantity])

    ddf = pd.DataFrame(data, columns=['Specification', 'Reference', 'Quantity'])
    # Item 열 추가
    def categorize_item(reference):
        reference = str(reference)
        if any(ref.startswith('R') for ref in reference.split(',')):
            return 'CHIP RES'
        elif any(ref.startswith('C') for ref in reference.split(',')):
            return 'CHIP CAP'
        elif any(ref.startswith('LD') for ref in reference.split(',')):
            return 'LED'
        elif any(ref.startswith('L') for ref in reference.split(',')):
            return 'Inductor'
        elif any(ref.startswith(('D', 'ED')) for ref in reference.split(',')):
            return 'Diode'
        elif any(ref.startswith('Q') for ref in reference.split(',')):
            return 'Transistor'
        elif any(ref.startswith(('X', 'Y')) for ref in reference.split(',')):
            return 'Crystal'
        elif any(ref.startswith('J') for ref in reference.split(',')):
            return 'Connector'
        elif any(ref.startswith('SW') for ref in reference.split(',')):
            return 'Switch'
        elif any(ref.startswith('U') for ref in reference.split(',')):
            return 'IC'
        else:
            return 'Others'

    ddf['Item'] = ddf['Reference'].apply(categorize_item)

    # Others 카테고리 항목을 마지막으로 이동
    others = ddf[ddf['Item'] == 'Others']
    ddf = pd.concat([ddf[ddf['Item'] != 'Others'], others]).reset_index(drop=True)

    def is_nc_resistor(spec):
      spec = str(spec).lower()
      return 'nc' in spec or 'no connection' in spec

    # NC 저항 식별 함수 추가
    def spec_strength(spec):
      if pd.isna(spec) or spec == '':
          return 0

      spec = str(spec).lower()

      if is_nc_resistor(spec):
          return float('inf')  # NC 저항은 가장 큰 값을 갖도록 설정

      multipliers = {'p': 1e-12, 'n': 1e-9, 'u': 1e-6, 'μ': 1e-6, 'k': 1e3, 'm': 1e6}

      for unit, multiplier in multipliers.items():
          if unit in spec:
              try:
                  return float(spec.split(unit)[0]) * multiplier
              except ValueError:
                  return 0

      try:
          return float(''.join(filter(str.isdigit, spec.split()[0])))
      except (ValueError, IndexError):
          return 0

    # 사용자 정의 정렬 키 함수 수정
    def custom_sort_key(row):
        item_order = {'CHIP CAP': 0, 'CHIP RES': 1, 'Inductor': 2, 'Diode': 3, 'Transistor': 4, 'Crystal': 5, 'IC': 6, 'Connector': 7, 'Switch': 8, 'Others': 9}
        item = row['Item']
        spec = row['Specification']

        if item == 'CHIP RES':
            if is_nc_resistor(spec):
                return (item_order[item], 1, spec_strength(spec))  # NC 저항은 CHIP RES 내에서 마지막에 정렬
            else:
                return (item_order[item], 0, spec_strength(spec))  # 일반 저항
        else:
            return (item_order.get(item, 10), 0, spec_strength(spec))

    # 데이터프레임에 정렬 키 열 추가
    ddf['sort_key'] = ddf.apply(custom_sort_key, axis=1)

    # 데이터프레임 정렬
    ddf = ddf.sort_values(by=['sort_key'])

    # 정렬 키 열 제거
    ddf = ddf.drop('sort_key', axis=1)

    # 열 순서 재배열
    ddf = ddf[['Item', 'Specification', 'Reference', 'Quantity']]

    # 인덱스 재설정
    ddf = ddf.reset_index(drop=True)

    # 새 워크북 생성
    wb = Workbook()
    ws = wb.active
    ws.title = "BOM"

    # 제목 추가
    ws['B1'] = "Bill of Materials (BOM)"
    ws['B1'].font = Font(bold=True, size=16)
    ws.merge_cells('B1:F1')

    # 수량 입력 칸 추가
    ws['G1'] = "생산수량:"
    ws['H1'] = 1  # 기본값 설정
    ws['G1'].font = Font(bold=True)
    ws['H1'].font = Font(bold=True, color="FF0000")  # 빨간색 설정
    ws['G1'].alignment = Alignment(horizontal='center', vertical='center')
    ws['H1'].alignment = Alignment(horizontal='center', vertical='center')

    # 날짜 추가
    today = date.today().strftime("%Y-%m-%d")
    ws['B2'] = f"Date: {today}"
    ws['B2'].font = Font(italic=True, bold=True)

    default_font = Font(bold=True)

    # 헤더 추가
    headers = ["No", "Item", "Specification", "Package(mm)", "Reference", "Quantity", "Total Qt", "In Stock", "Required", "Remarks"]
    for c, header in enumerate(headers, start=2):
        ws.cell(row=3, column=c, value=header)

    # 데이터프레임을 워크시트에 쓰기 (4행부터 시작)
    for r, row in enumerate(ddf.values, start=4):
        no_cell = ws.cell(row=r, column=2, value=r-3)
        no_cell.font = default_font
        for c, value in enumerate(row, start=3):
            if pd.notna(value):  # NaN 값을 건너뛰기
                if c < 5:  # Package 열 이전까지는 그대로 입력
                    data_cell = ws.cell(row=r, column=c, value=value)
                    data_cell.font = default_font
                elif c == 5:  # Package 열에 빈 값 입력
                    data_cell = ws.cell(row=r, column=c, value="")
                    data_cell.font = default_font
                    reference_cell = ws.cell(row=r, column=c+1, value=str(value).replace('"', ''))  # 큰따옴표 제거
                    reference_cell.font = default_font
                else:  # 나머지 열들도 한 칸씩 뒤로
                    data_cell = ws.cell(row=r, column=c+1, value=value)
                    data_cell.font = default_font

    # 제목 변경
    file_name_without_extension = os.path.splitext(file_name)[0]
    ws['B1'] = file_name_without_extension
    ws['B1'].font = Font(bold=True, size=16)
    ws.merge_cells('B1:F1')

    # Others 카테고리 처리
    others_rows = [r for r, row in enumerate(ddf.values, start=4) if row[0] == 'Others']
    if others_rows:
      for r in others_rows:
          ws.cell(row=r, column=3, value="Others")
      ws.merge_cells(f'C{others_rows[0]}:C{others_rows[-1]}')
      ws.cell(row=others_rows[0], column=3).alignment = Alignment(horizontal='center', vertical='center')

    # 같은 카테고리 셀 병합
    current_category = None
    start_row = 4

    for row in range(4, ws.max_row + 1):
        category = ws.cell(row=row, column=3).value
        if category != current_category:
            if current_category is not None:
                # 이전 카테고리 셀 병합
                if start_row < row - 1:
                    ws.merge_cells(f'C{start_row}:C{row-1}')
            current_category = category
            start_row = row

    # 마지막 카테고리 셀 병합
    if start_row < ws.max_row:
        ws.merge_cells(f'C{start_row}:C{ws.max_row}')

    # 병합된 셀의 정렬 조정
    for merged_range in ws.merged_cells.ranges:
        if merged_range.min_col == 3:  # C열(카테고리 열)
            ws.cell(row=merged_range.min_row, column=3).alignment = Alignment(horizontal='center', vertical='center')

    # Total Quantity 계산 공식 추가
    for row in range(4, ws.max_row + 1):
        quantity_cell = ws.cell(row=row, column=7)
        total_quantity_cell = ws.cell(row=row, column=8)
        total_quantity_cell.value = f"={quantity_cell.coordinate}*$H$1"

    # Required 계산 공식 추가
    for row in range(4, ws.max_row + 1):
        total_quantity_cell = ws.cell(row=row, column=8)
        in_stock_cell = ws.cell(row=row, column=9)
        required_cell = ws.cell(row=row, column=10)
        required_cell.value = f"=MAX(0, {total_quantity_cell.coordinate}-{in_stock_cell.coordinate})"

    # 스타일 설정
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="B4C6E7", end_color="B4C6E7", fill_type="solid")
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    thick_border = Border(left=Side(style='thick'), right=Side(style='thick'), top=Side(style='thick'), bottom=Side(style='thick'))

    category_colors = {
        "CHIP RES": "FFC7CE",
        "CHIP CAP": "C6EFCE",
        "Diode": "FFEB9C",
        "Transistor": "FFEB9C",
        "Crystal": "FFEB9C",
        "Connector": "BDD7EE",
        "Switch": "BDD7EE",
        "IC": "FFEB9C",
        "Inductor": "C6EFCE",
        "Others": "D9E1F2"
    }

    # 스타일 적용
    for row in ws.iter_rows(min_row=3, max_row=ws.max_row, min_col=2, max_col=ws.max_column):
        for cell in row:
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center', vertical='center')

            if cell.row == 3 or cell.column == 2:  # 헤더 행이거나 No 열인 경우
                cell.font = header_font
                cell.fill = header_fill
            else:
                category = ws.cell(row=cell.row, column=3).value
                if category in category_colors:
                    fill_color = category_colors[category]
                    cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")

                if cell.column in [4, 5, 6]:  # D열, E열, F열 정렬 해제
                    cell.alignment = Alignment(horizontal='left', vertical='center')

    # A열의 스타일 제거
    for cell in ws['A']:
        cell.border = None
        cell.fill = PatternFill(fill_type=None)

    # 외곽 테두리 두껍게 설정
    last_row = ws.max_row
    last_col = ws.max_column

    for col in range(2, last_col + 1):
        ws.cell(row=4, column=col).border = Border(top=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), bottom=Side(style='thin'))
        ws.cell(row=last_row, column=col).border = Border(bottom=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'))

    for row in range(4, last_row + 1):
        ws.cell(row=row, column=2).border = Border(left=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), right=Side(style='thin'))
        ws.cell(row=row, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'))

    # 모서리 셀 테두리 수정
    ws.cell(row=4, column=2).border = Border(left=Side(style='thick'), top=Side(style='thick'), right=Side(style='thin'), bottom=Side(style='thin'))
    ws.cell(row=4, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thick'), left=Side(style='thin'), bottom=Side(style='thin'))

    # 열 너비 조정
    width = 10  # 기본 너비
    for column in ws.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column_letter].width = adjusted_width
        ws.column_dimensions['B'].width = 6  # A열의 너비를 6으로 설정
        ws.column_dimensions['F'].width = 40  # E열의 너비를 40으로 설정
        ws.column_dimensions['G'].width = width  # F열의 너비를 10으로 설정
        ws.column_dimensions['H'].width = width  # E열의 너비를 10으로 설정
        ws.column_dimensions['I'].width = width  # E열의 너비를 10으로 설정
        ws.column_dimensions['J'].width = width  # E열의 너비를 10으로 설정
        ws.column_dimensions['K'].width = 25  # J열의 너비를 16으로 설정

    # 엑셀 파일 저장
    output_file_name = os.path.splitext(file_name)[0] + ".xlsx"
    wb.save(output_file_name)

Saving NIDS_NEW_BREATHALYZER_V11_20240813.txt to NIDS_NEW_BREATHALYZER_V11_20240813 (1).txt
Processing file "NIDS_NEW_BREATHALYZER_V11_20240813 (1).txt" with length 3194 bytes


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 3033: invalid start byte

In [ ]:
answer ='''

 Category,Specification,Reference,Quantity
Capacitor,1uF,"C43,C46",2
Capacitor,10nF,C45,1
Capacitor,100nF,"C44,C47,C48,C50,C51,C52,C53",7
Resistor,0,"R79,R99",2
Resistor,0R,"R88,R91",2
Resistor,10/1608,R93,1
Resistor,100,"R76,R77,R78",3
Resistor,1K,"R89,R92,R105",3
Resistor,20K,R83,1
Resistor,47K,R80,1
Resistor,330,"R87,R90",2
Resistor,1M,R82,1
Resistor,100K,"R95,R96,R97,R100,R101",5
Resistor,330K,R102,1
Resistor,620K,R94,1
NC,NC,"C49,R75,R81,R84,R85,R86,R98,R103,R104,R106",10
Diode,LESD5D5.0CT1G,"D15,D16,D17",3
Diode,ESD5Z3.3T1G,ED2,1
Connector,Header_2mm_5P,J6,1
Connector,B2B_2.0MM_7P_MALE,J7,1
Inductor,CP100508T-101Y,L7,1
Mechanical,HOLE_1.7,"MT10,MT11",2
Transistor,WNM3003,Q4,1
IC,XMSM0C1104SDYYR_SOT23-16P,U13,1
IC,AZ431LANTR-E1,U14,1
IC,LTC2050CS5#TRMPBF,U15,1
IC,MCP6001T-E/OT,U16,1
Sensor,ME2-C2H5OH-13×13,U17,1
IC,SGM8552XMS8G/TR,U18,1 '''


print("Claude의 답변:", answer)
import pandas as pd
import io
import re
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.datavalidation import DataValidation
from datetime import date
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment, Color, Border, Side



# Attempt to read the CSV, handling potential errors
try:
    df = pd.read_csv(io.StringIO(answer))
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
    # Print the problematic lines for debugging
    for i, line in enumerate(answer.split('\n')):
        if i >= 13:  # Print lines around line 15
            print(f"Line {i+1}: {line}")
    # Optionally, you can try to manually fix the CSV data here before reading it again.

# 새 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "BOM"

# 제목 추가
ws['B1'] = f"{extract_sub}"
ws['B1'].font = Font(bold=True, size=16)
ws.merge_cells('B1:F1')

# 수량 입력 칸 추가
ws['G1'] = "생산수량:"
ws['H1'] = 1  # 기본값 설정
ws['G1'].font = Font(bold=True)
ws['H1'].font = Font(bold=True)
ws['B1'].alignment = Alignment(horizontal='center', vertical='center')  # B1 셀 가운데 정렬
ws['H1'].font = Font(bold=True, color="FF0000")  # 빨간색 설정
ws['H1'].alignment = Alignment(horizontal='center', vertical='center')  # H1 셀 가운데 정렬
ws['G1'].alignment = Alignment(horizontal='center', vertical='center')  # G1 셀 가운데 정렬

# 날짜 추가
today = date.today().strftime("%Y-%m-%d")
ws['B2'] = f"Date: {today}"
ws['B2'].font = Font(italic=True, bold=True)

default_font = Font(bold=True)

# 헤더 추가
headers = ["No", "Category", "Specification", "Package(mm)", "Reference", "Quantity", "Total Qt", "In Stock", "Required", "Remarks"]
for c, header in enumerate(headers, start=2):
    ws.cell(row=4, column=c, value=header)

# 데이터프레임을 워크시트에 쓰기 (5행부터 시작)
for r, row in enumerate(df.values, start=5):
    no_cell = ws.cell(row=r, column=2, value=r-4)
    no_cell.font = default_font
    for c, value in enumerate(row, start=3):
        if c < 5:  # Package 열 이전까지는 그대로 입력
            data_cell = ws.cell(row=r, column=c, value=value)
            data_cell.font = default_font
        elif c == 5:  # Package 열에 빈 값 입력
            data_cell = ws.cell(row=r, column=c, value="")
            data_cell.font = default_font
            reference_cell = ws.cell(row=r, column=c+1, value=value)
            reference_cell.font = default_font
        else:  # 나머지 열들도 한 칸씩 뒤로
            data_cell = ws.cell(row=r, column=c+1, value=value)
            data_cell.font = default_font

# 같은 카테고리 셀 병합
current_category = None
start_row = 5

for row in range(5, ws.max_row + 1):
    category = ws.cell(row=row, column=3).value
    if category != current_category:
        if current_category is not None:
            # 이전 카테고리 셀 병합
            if start_row < row - 1:
                ws.merge_cells(f'C{start_row}:C{row-1}')
        current_category = category
        start_row = row

# 마지막 카테고리 셀 병합
if start_row < ws.max_row:
    ws.merge_cells(f'C{start_row}:C{ws.max_row}')

# 병합된 셀의 정렬 조정
for merged_range in ws.merged_cells.ranges:
    if merged_range.min_col == 3:  # C열(카테고리 열)
        ws.cell(row=merged_range.min_row, column=3).alignment = Alignment(horizontal='center', vertical='center')

# Total Quantity 계산 공식 추가
for row in range(5, len(df) + 5):
    quantity_cell = ws.cell(row=row, column=7)
    total_quantity_cell = ws.cell(row=row, column=8)
    total_quantity_cell.value = f"={quantity_cell.coordinate}*$H$1"
    total_quantity_cell.font = Font(bold=True)  # 굵은 글씨 적용


# Required 계산 공식 추가
for row in range(5, len(df) + 5):
    total_quantity_cell = ws.cell(row=row, column=8)
    in_stock_cell = ws.cell(row=row, column=9)
    required_cell = ws.cell(row=row, column=10)
    required_cell.value = f"=MAX(0, {total_quantity_cell.coordinate}-{in_stock_cell.coordinate})"
    required_cell.font = Font(bold=True)  # 굵은 글씨 적용

# 스타일 설정 (변경된 부분)
header_font = Font(bold=True)
header_fill = PatternFill(start_color="B4C6E7", end_color="B4C6E7", fill_type="solid")
#thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
#thick_border = Border(left=Side(style='thick'), right=Side(style='thick'), top=Side(style='thick'), bottom=Side(style='thick'))

category_colors = {
    "Resistor": "FFD9D9",      # 연한 빨강
    "Capacitor": "D9FFD9",     # 연한 초록
    "Diode": "FFE6B3",         # 연한 주황
    "Transistor": "B3E6FF",    # 연한 파랑
    "Crystal": "E6B3FF",       # 연한 보라
    "LED": "FFFFB3",           # 연한 노랑
    "Inductor": "FFB3D9",      # 연한 분홍
    "Bead": "D9E6FF",          # 매우 연한 파랑
    "ESD Protection": "E6FFB3", # 연한 라임
    "IC": "B3FFE6",            # 연한 민트
    "Connector": "FFD9B3",     # 연한 복숭아
    "Switch": "D9FFE6",        # 연한 청록
    "Buzzer": "E6D9FF",        # 연한 라벤더
    "Test Point": "FFE6E6",  # 매우 연한 분홍
    "TP": "FFE6E6",  # 매우 연한 분홍,
    "Others": "E6E6E6"         # 연한 회색
}


# 스타일 적용 (변경된 부분)
for row in ws.iter_rows(min_row=4, max_row=ws.max_row, min_col=2, max_col=ws.max_column):
    for cell in row:
        thin_border = Border(left=Side(style='thin'),
                     right=Side(style='thin'),
                     top=Side(style='thin'),
                     bottom=Side(style='thin'))

        thick_border = Border(left=Side(style='thick'),
                      right=Side(style='thick'),
                      top=Side(style='thick'),
                      bottom=Side(style='thick'))
        cell.border = thin_border
        cell.alignment = Alignment(horizontal='center', vertical='center')

        if cell.row == 4 or cell.column == 2:  # 헤더 행이거나 No 열인 경우
            cell.font = header_font
            cell.fill = header_fill
        else:
            category = ws.cell(row=cell.row, column=3).value
            if category in category_colors:
                fill_color = category_colors[category]
                cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")

            if cell.column in [4, 5, 6]:  # D열, E열, F열 정렬 해제
                cell.alignment = Alignment(horizontal='left', vertical='center')

# A열의 스타일 제거
for cell in ws['A']:
    cell.border = None
    cell.fill = PatternFill(fill_type=None)

# 외곽 테두리 두껍게 설정 (새로 추가된 부분)
last_row = ws.max_row
last_col = ws.max_column

for col in range(2, last_col + 1):
    ws.cell(row=4, column=col).border = Border(top=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), bottom=Side(style='thin'))
    ws.cell(row=last_row, column=col).border = Border(bottom=Side(style='thick'), left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'))

for row in range(4, last_row + 1):
    ws.cell(row=row, column=2).border = Border(left=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), right=Side(style='thin'))
    ws.cell(row=row, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thin'), bottom=Side(style='thin'), left=Side(style='thin'))

# 모서리 셀 테두리 수정 (새로 추가된 부분)
ws.cell(row=4, column=2).border = Border(left=Side(style='thick'), top=Side(style='thick'), right=Side(style='thin'), bottom=Side(style='thin'))
ws.cell(row=4, column=last_col).border = Border(right=Side(style='thick'), top=Side(style='thick'), left=Side(style='thin'), bottom=Side(style='thin'))
ws.cell(row=last_row, column=2).border = Border(left=Side(style='thick'), bottom=Side(style='thick'), right=Side(style='thin'), top=Side(style='thin'))
ws.cell(row=last_row, column=last_col).border = Border(right=Side(style='thick'), bottom=Side(style='thick'), left=Side(style='thin'), top=Side(style='thin'))

# 열 너비 조정
width = 10  # 기본 너비
for column in ws.columns:
    max_length = 0
    column_letter = get_column_letter(column[0].column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column_letter].width = adjusted_width
    ws.column_dimensions['B'].width = 6  # A열의 너비를 6으로 설정
    ws.column_dimensions['F'].width = 40  # E열의 너비를 40으로 설정
    ws.column_dimensions['G'].width = width  # F열의 너비를 10으로 설정
    ws.column_dimensions['H'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['I'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['J'].width = width  # E열의 너비를 10으로 설정
    ws.column_dimensions['K'].width = 25  # J열의 너비를 16으로 설정


# 'resister' 문자열을 찾아 'chip capacitor'로 변경
for row in ws.iter_rows():
    for cell in row:
        if cell.value == 'Resistor':
            cell.value = 'Chip Res'
        if cell.value == 'Capacitor':
            cell.value = 'Chip Cap'


# 특수 문자를 언더스코어로 대체
extract_sub = re.sub(r'[<>:"/\\|?*&]', '_', extract_sub)

# Production Quantity 셀에 데이터 유효성 검사 추가
dv = DataValidation(type="whole", operator="greaterThan", formula1=0)
dv.error = "Please enter a positive integer"
dv.errorTitle = "Invalid Input"
ws.add_data_validation(dv)
dv.add('H1')

# 파일 저장
excel_file = f'{extract_sub}.xlsx'
wb.save(excel_file)

print(f"Excel 파일이 생성되었습니다: {extract_sub}xlsx")

Claude의 답변: 
  
 Category,Specification,Reference,Quantity
Capacitor,1uF,"C43,C46",2
Capacitor,10nF,C45,1
Capacitor,100nF,"C44,C47,C48,C50,C51,C52,C53",7
Resistor,0,"R79,R99",2
Resistor,0R,"R88,R91",2
Resistor,10/1608,R93,1
Resistor,100,"R76,R77,R78",3
Resistor,1K,"R89,R92,R105",3
Resistor,20K,R83,1
Resistor,47K,R80,1
Resistor,330,"R87,R90",2
Resistor,1M,R82,1
Resistor,100K,"R95,R96,R97,R100,R101",5
Resistor,330K,R102,1
Resistor,620K,R94,1
NC,NC,"C49,R75,R81,R84,R85,R86,R98,R103,R104,R106",10
Diode,LESD5D5.0CT1G,"D15,D16,D17",3
Diode,ESD5Z3.3T1G,ED2,1
Connector,Header_2mm_5P,J6,1
Connector,B2B_2.0MM_7P_MALE,J7,1
Inductor,CP100508T-101Y,L7,1
Mechanical,HOLE_1.7,"MT10,MT11",2
Transistor,WNM3003,Q4,1
IC,XMSM0C1104SDYYR_SOT23-16P,U13,1
IC,AZ431LANTR-E1,U14,1
IC,LTC2050CS5#TRMPBF,U15,1
IC,MCP6001T-E/OT,U16,1
Sensor,ME2-C2H5OH-13×13,U17,1
IC,SGM8552XMS8G/TR,U18,1 
Excel 파일이 생성되었습니다: NIDS_New_Breathalyzer__xlsx
